In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_11 import *

## Serializing the model

In [3]:
path = untar_data(URLs.IMAGEWOOF_160)

File downloaded is broken. Remove /storage/archive/imagewoof2-160.tgz and try again.


In [5]:
size = 128
bs = 64

tfms = [make_rgb, RandomResizedCrop(size, scale=(0.35,1)), np_to_float, PilRandomFlip()]
val_tfms = [make_rgb, CenterCrop(size), np_to_float]
il = ImageList.from_files(path, tfms=tfms)
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='val'))
ll = label_by_func(sd, parent_labeler, proc_y=CategoryProcessor())
ll.valid.x.tfms = val_tfms
data = ll.to_databunch(bs, c_in=3, c_out=10, num_workers=8)

In [6]:
len(il)

12954

In [7]:
loss_func = LabelSmoothingCrossEntropy()
opt_func = adam_opt(mom=0.9, mom_sqr=0.99, eps=1e-6, wd=1e-2)

In [8]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [9]:
def sched_1cycle(lr, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
    phases = create_phases(pct_start)
    sched_lr  = combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
    sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
    return [ParamScheduler('lr', sched_lr),
            ParamScheduler('mom', sched_mom)]

In [10]:
lr = 3e-3
pct_start = 0.5
cbsched = sched_1cycle(lr, pct_start)

In [11]:
learn.fit(10, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.132365,0.241108,2.116131,0.271316,00:42
1,2.021092,0.301053,2.215078,0.259863,00:42
2,1.938759,0.343712,2.651702,0.231357,00:43
3,1.871181,0.380942,2.392920,0.280733,00:42
4,1.791140,0.420831,2.139304,0.330618,00:42
5,1.697227,0.463601,2.216897,0.333673,00:42
6,1.599190,0.521108,1.759520,0.454569,00:42
7,1.481115,0.573740,1.459214,0.584373,00:42
8,1.373891,0.625042,1.384149,0.615169,00:42
9,1.316471,0.654072,1.335574,0.645457,00:42


In [34]:
st = learn.model.state_dict()

In [35]:
type(st)

collections.OrderedDict

In [36]:
', '.join(st.keys())

'0.0.weight, 0.1.weight, 0.1.bias, 0.1.running_mean, 0.1.running_var, 0.1.num_batches_tracked, 1.0.weight, 1.1.weight, 1.1.bias, 1.1.running_mean, 1.1.running_var, 1.1.num_batches_tracked, 2.0.weight, 2.1.weight, 2.1.bias, 2.1.running_mean, 2.1.running_var, 2.1.num_batches_tracked, 4.0.convs.0.0.weight, 4.0.convs.0.1.weight, 4.0.convs.0.1.bias, 4.0.convs.0.1.running_mean, 4.0.convs.0.1.running_var, 4.0.convs.0.1.num_batches_tracked, 4.0.convs.1.0.weight, 4.0.convs.1.1.weight, 4.0.convs.1.1.bias, 4.0.convs.1.1.running_mean, 4.0.convs.1.1.running_var, 4.0.convs.1.1.num_batches_tracked, 4.1.convs.0.0.weight, 4.1.convs.0.1.weight, 4.1.convs.0.1.bias, 4.1.convs.0.1.running_mean, 4.1.convs.0.1.running_var, 4.1.convs.0.1.num_batches_tracked, 4.1.convs.1.0.weight, 4.1.convs.1.1.weight, 4.1.convs.1.1.bias, 4.1.convs.1.1.running_mean, 4.1.convs.1.1.running_var, 4.1.convs.1.1.num_batches_tracked, 5.0.convs.0.0.weight, 5.0.convs.0.1.weight, 5.0.convs.0.1.bias, 5.0.convs.0.1.running_mean, 5.0.convs

In [37]:
st['10.bias']

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [38]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)

It's also possible to save the whole model, including the architecture, but it gets quite fiddly and we don't recommend it. Instead, just save the parameters, and recreate the model directly.

In [39]:
torch.save(st, mdl_path/'iw5')

## Pets

In [12]:
pets = untar_data(URLs.PETS)

In [13]:
pets.ls()

[Path('/storage/data/oxford-iiit-pet/images'),
 Path('/storage/data/oxford-iiit-pet/annotations')]

In [14]:
pets_path = pets/'images'

In [15]:
il = ImageList.from_files(pets_path, tfms=tfms)

In [16]:
il

ImageList (7390 items)
[Path('/storage/data/oxford-iiit-pet/images/miniature_pinscher_199.jpg'), Path('/storage/data/oxford-iiit-pet/images/newfoundland_183.jpg'), Path('/storage/data/oxford-iiit-pet/images/pomeranian_90.jpg'), Path('/storage/data/oxford-iiit-pet/images/pomeranian_102.jpg'), Path('/storage/data/oxford-iiit-pet/images/japanese_chin_74.jpg'), Path('/storage/data/oxford-iiit-pet/images/yorkshire_terrier_45.jpg'), Path('/storage/data/oxford-iiit-pet/images/chihuahua_34.jpg'), Path('/storage/data/oxford-iiit-pet/images/american_pit_bull_terrier_150.jpg'), Path('/storage/data/oxford-iiit-pet/images/wheaten_terrier_160.jpg'), Path('/storage/data/oxford-iiit-pet/images/staffordshire_bull_terrier_91.jpg')...]
Path: /storage/data/oxford-iiit-pet/images

In [17]:
#export
def random_splitter(fn, p_valid): return random.random() < p_valid

In [18]:
random.seed(42)

In [19]:
sd = SplitData.split_by_func(il, partial(random_splitter, p_valid=0.1))

In [20]:
sd

SplitData
Train: ImageList (6667 items)
[Path('/storage/data/oxford-iiit-pet/images/miniature_pinscher_199.jpg'), Path('/storage/data/oxford-iiit-pet/images/pomeranian_90.jpg'), Path('/storage/data/oxford-iiit-pet/images/pomeranian_102.jpg'), Path('/storage/data/oxford-iiit-pet/images/japanese_chin_74.jpg'), Path('/storage/data/oxford-iiit-pet/images/yorkshire_terrier_45.jpg'), Path('/storage/data/oxford-iiit-pet/images/chihuahua_34.jpg'), Path('/storage/data/oxford-iiit-pet/images/wheaten_terrier_160.jpg'), Path('/storage/data/oxford-iiit-pet/images/Siamese_42.jpg'), Path('/storage/data/oxford-iiit-pet/images/basset_hound_80.jpg'), Path('/storage/data/oxford-iiit-pet/images/samoyed_124.jpg')...]
Path: /storage/data/oxford-iiit-pet/images
Valid: ImageList (723 items)
[Path('/storage/data/oxford-iiit-pet/images/newfoundland_183.jpg'), Path('/storage/data/oxford-iiit-pet/images/american_pit_bull_terrier_150.jpg'), Path('/storage/data/oxford-iiit-pet/images/staffordshire_bull_terrier_91.j

In [21]:
n = il.items[0].name; n

'miniature_pinscher_199.jpg'

In [22]:
re.findall(r'^(.*)_\d+.jpg$', n)[0]

'miniature_pinscher'

In [23]:
def pet_labeler(fn): return re.findall(r'^(.*)_\d+.jpg$', fn.name)[0]

In [24]:
proc = CategoryProcessor()

In [25]:
ll = label_by_func(sd, pet_labeler, proc_y=proc)

In [26]:
', '.join(proc.vocab)

'miniature_pinscher, pomeranian, japanese_chin, yorkshire_terrier, chihuahua, wheaten_terrier, Siamese, basset_hound, samoyed, saint_bernard, Persian, english_setter, Russian_Blue, beagle, shiba_inu, great_pyrenees, german_shorthaired, scottish_terrier, Birman, Abyssinian, pug, english_cocker_spaniel, boxer, newfoundland, Maine_Coon, havanese, american_bulldog, Ragdoll, staffordshire_bull_terrier, Bombay, Sphynx, Egyptian_Mau, keeshond, leonberger, american_pit_bull_terrier, British_Shorthair, Bengal'

In [27]:
ll.valid.x.tfms = val_tfms

In [28]:
c_out = len(proc.vocab)

In [29]:
data = ll.to_databunch(bs, c_in=3, c_out=c_out, num_workers=8)

In [30]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [31]:
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.489200,0.085196,3.407722,0.087137,01:10
1,3.289816,0.131543,3.522285,0.109267,00:30
2,3.071774,0.190790,3.322322,0.147994,00:30
3,2.786011,0.275686,2.690252,0.301521,00:30
4,2.464478,0.386981,2.486617,0.376210,00:30


## Custom head

In [40]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [41]:
st = torch.load(mdl_path/'iw5')

In [42]:
m = learn.model

In [43]:
m.load_state_dict(st)

<All keys matched successfully>

In [44]:
cut = next(i for i,o in enumerate(m.children()) if isinstance(o,nn.AdaptiveAvgPool2d))
m_cut = m[:cut]

In [45]:
xb,yb = get_batch(data.valid_dl, learn)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [46]:
pred = m_cut(xb)

In [47]:
pred.shape

torch.Size([128, 512, 4, 4])

In [48]:
ni = pred.shape[1]

In [49]:
#export
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=1):
        super().__init__()
        self.output_size = sz
        self.ap = nn.AdaptiveAvgPool2d(sz)
        self.mp = nn.AdaptiveMaxPool2d(sz)
    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)

In [50]:
nh = 40

m_new = nn.Sequential(
    m_cut, AdaptiveConcatPool2d(), Flatten(),
    nn.Linear(ni*2, data.c_out))

In [51]:
learn.model = m_new

In [52]:
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.472536,0.087896,3.543937,0.113416,00:30
1,3.305072,0.140843,5.636245,0.051176,00:30
2,3.066183,0.203990,2.980818,0.199170,00:30
3,2.692182,0.306285,2.761807,0.280775,00:30
4,2.346435,0.427029,2.367594,0.409405,00:30


## adapt_model and gradual unfreezing

In [53]:
def adapt_model(learn, data):
    cut = next(i for i,o in enumerate(learn.model.children())
               if isinstance(o,nn.AdaptiveAvgPool2d))
    m_cut = learn.model[:cut]
    xb,yb = get_batch(data.valid_dl, learn)
    pred = m_cut(xb)
    ni = pred.shape[1]
    m_new = nn.Sequential(
        m_cut, AdaptiveConcatPool2d(), Flatten(),
        nn.Linear(ni*2, data.c_out))
    learn.model = m_new

In [54]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))

<All keys matched successfully>

In [55]:
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [56]:
for p in learn.model[0].parameters(): p.requires_grad_(False)

In [57]:
learn.fit(3, sched_1cycle(1e-2, 0.5))

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.826415,0.064647,5.492538,0.053942,00:12
1,5.423944,0.086546,4.405218,0.116183,00:12
2,3.453576,0.163342,3.207391,0.172891,00:12


In [58]:
for p in learn.model[0].parameters(): p.requires_grad_(True)

In [59]:
learn.fit(5, cbsched, reset_opt=True)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.103951,0.196040,3.358190,0.145228,00:30
1,3.185475,0.186141,3.500953,0.153527,00:31
2,3.046574,0.218539,3.363775,0.157676,00:30
3,2.681516,0.311384,2.599098,0.309820,00:31
4,2.328670,0.429129,2.367522,0.417704,00:30


## Batch norm transfer

In [60]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [61]:
def apply_mod(m, f):
    f(m)
    for l in m.children(): apply_mod(l, f)

def set_grad(m, b):
    if isinstance(m, (nn.Linear,nn.BatchNorm2d)): return
    if hasattr(m, 'weight'):
        for p in m.parameters(): p.requires_grad_(b)

In [62]:
apply_mod(learn.model, partial(set_grad, b=False))

In [63]:
learn.fit(3, sched_1cycle(1e-2, 0.5))

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.743845,0.068697,3.971596,0.096819,00:23
1,3.360024,0.131693,3.225025,0.157676,00:22
2,3.063680,0.201140,3.048254,0.218534,00:23


In [64]:
apply_mod(learn.model, partial(set_grad, b=True))

In [65]:
learn.fit(5, cbsched, reset_opt=True)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.969525,0.231288,3.264585,0.157676,00:30
1,2.869676,0.255437,3.136619,0.175657,00:30
2,2.721586,0.292035,2.972492,0.217151,00:30
3,2.437001,0.392830,2.467385,0.352697,00:31
4,2.105749,0.509825,2.182819,0.459198,00:30


Pytorch already has an `apply` method we can use:

In [ ]:
learn.model.apply(partial(set_grad, b=False));

## Discriminative LR and param groups

In [66]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [67]:
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [68]:
def bn_splitter(m):
    def _bn_splitter(l, g1, g2):
        if isinstance(l, nn.BatchNorm2d): g2 += l.parameters()
        elif hasattr(l, 'weight'): g1 += l.parameters()
        for ll in l.children(): _bn_splitter(ll, g1, g2)
        
    g1,g2 = [],[]
    _bn_splitter(m[0], g1, g2)
    
    g2 += m[1:].parameters()
    return g1,g2

In [69]:
a,b = bn_splitter(learn.model)

In [70]:
test_eq(len(a)+len(b), len(list(m.parameters())))

In [71]:
Learner.ALL_CBS

{'after_backward',
 'after_batch',
 'after_cancel_batch',
 'after_cancel_epoch',
 'after_cancel_train',
 'after_epoch',
 'after_fit',
 'after_loss',
 'after_pred',
 'after_step',
 'begin_batch',
 'begin_epoch',
 'begin_fit',
 'begin_validate'}

In [72]:
#export
from types import SimpleNamespace
cb_types = SimpleNamespace(**{o:o for o in Learner.ALL_CBS})

In [73]:
cb_types.after_backward

'after_backward'

In [74]:
#export
class DebugCallback(Callback):
    _order = 999
    def __init__(self, cb_name, f=None): self.cb_name,self.f = cb_name,f
    def __call__(self, cb_name):
        if cb_name==self.cb_name:
            if self.f: self.f(self.run)
            else:      set_trace()

In [75]:
#export
def sched_1cycle(lrs, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
    phases = create_phases(pct_start)
    sched_lr  = [combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
                 for lr in lrs]
    sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
    return [ParamScheduler('lr', sched_lr),
            ParamScheduler('mom', sched_mom)]

In [76]:
disc_lr_sched = sched_1cycle([0,3e-2], 0.5)

In [77]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func,
                    c_out=10, norm=norm_imagenette, splitter=bn_splitter)

learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [78]:
def _print_det(o): 
    print (len(o.opt.param_groups), o.opt.hypers)
    raise CancelTrainException()

learn.fit(1, disc_lr_sched + [DebugCallback(cb_types.after_batch, _print_det)])

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


2 [{'mom': 0.9499999999999997, 'mom_sqr': 0.99, 'eps': 1e-06, 'wd': 0.01, 'lr': 0.0, 'sqr_mom': 0.99}, {'mom': 0.9499999999999997, 'mom_sqr': 0.99, 'eps': 1e-06, 'wd': 0.01, 'lr': 0.0030000000000000512, 'sqr_mom': 0.99}]


In [79]:
learn.fit(3, disc_lr_sched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.838603,0.079046,3.386166,0.109267,00:58
1,3.308230,0.123744,3.322330,0.128631,00:30
2,3.110452,0.181791,3.047529,0.196404,00:30


In [80]:
disc_lr_sched = sched_1cycle([1e-3,1e-2], 0.3)

In [81]:
learn.fit(5, disc_lr_sched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.949128,0.227839,3.188183,0.177040,00:30
1,2.925633,0.231288,3.123337,0.201936,00:30
2,2.698109,0.310484,2.955657,0.236515,00:30
3,2.450850,0.379331,2.464325,0.372061,00:30
4,2.241561,0.459877,2.382710,0.388658,00:30


## Export

In [ ]:
!./notebook2script.py 11a_transfer_learning.ipynb